<a href="https://colab.research.google.com/github/Method-for-Software-System-Development/Cloud_Computing/blob/develop/logic/chatbot_controller.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ---- Imports ----
import time
import difflib
from datetime import datetime

import google.generativeai as genai
from google.colab import userdata

# Load other project modules using importnb
from importnb import Notebook
with Notebook():
    import mqqt_sim_indoor as indoor
    import mqqt_sim_outdoor as outdoor
    import user_controller as uc
    import FireBase as fb

# Gemini API key (ensure this is set in your Colab secrets)
GEMINI_API_KEY = userdata.get("GOOGLE_API_KEY")

In [ ]:
"""
Sensor Meta Table
-----------------
Defines metadata for all sensors in the OptiLine system.
This information is used to answer questions about sensor functions, valid value ranges, and sensor descriptions.
"""

SENSORS_META = {
    "temperature": {
        "label": "Temperature",
        "units": "°C",
        "locations": ["indoor", "outdoor"],
        "description": "Measures ambient air temperature.",
        "normal_range": [18, 26],
        "details": "Temperature sensors are used to monitor the climate in the production environment and ensure optimal conditions."
    },
    "humidity": {
        "label": "Humidity",
        "units": "%",
        "locations": ["indoor", "outdoor"],
        "description": "Measures relative air humidity.",
        "normal_range": [30, 60],
        "details": "Humidity sensors help prevent issues such as condensation, corrosion, or material degradation."
    },
    "pressure": {
        "label": "Pressure",
        "units": "hPa",
        "locations": ["indoor"],
        "description": "Measures atmospheric pressure.",
        "normal_range": [1000, 1025],
        "details": "Pressure sensors provide data that can be used for weather analysis or to ensure safety standards in the facility."
    },
    "dlight": {
        "label": "Dlight",
        "units": "Lux",
        "locations": ["outdoor"],
        "description": "Measures light intensity (illuminance).",
        "normal_range": [10000, 50000],
        "details": "Dlight sensors can be used to monitor daylight exposure, useful for environmental analysis."
    },
    "distance": {
        "label": "Distance",
        "units": "mm",
        "locations": ["indoor"],
        "description": "Ultrasonic sensor detects distance changes (e.g., movement nearby).",
        "normal_range": [0, 500],
        "details": "Distance sensors are used for safety, automation, and detecting human or object presence."
    }
}


In [ ]:
"""
User Guide, FAQ, and System Context
-----------------------------------
This context is included in every prompt to Gemini, providing the model with up-to-date knowledge about the OptiLine Dashboard,
the CIM & Robotics Laboratory, available sensors, and operational policies.
"""

OPTI_BOT_CONTEXT = """
Welcome to the OptiLine Dashboard – a real-time cloud-based interface for engineers in the CIM & Robotics Lab.

Main Features:
• Live Sensors Dashboard – Monitor real-time data from indoor and outdoor sensors.
• Statistics Panel – Analyze historical sensor data with interactive plots.
• MQTT Search Engine – Search technical documentation from MQTT.org.
• Fault Simulator – Practice troubleshooting simulated malfunctions.
• User Directory – View contact information for system users.
• Leaderboard – Track your performance points relative to other engineers.

Available Sensors:
- Temperature (°C): Measures ambient temperature (indoor & outdoor)
- Humidity (%): Measures relative humidity (indoor & outdoor)
- Pressure (hPa): Measures atmospheric pressure (indoor)
- Dlight (Lux): Measures light intensity (outdoor)
- Distance (mm): Ultrasonic sensor for movement detection (indoor)
All sensors stream data in real time via MQTT protocol.

How to Earn Points:
- Complete fault simulator challenges.
- More complex faults award higher scores.
- Time efficiency matters: faster repairs = better ranking.
- You must complete all simulator steps for points; no partial credit.
- Your total score determines your leaderboard rank.

FAQ:
Q: Can I skip steps in the Fault Simulator and still get points?
A: No. You must complete all required steps before submitting. Partial credit is not available.

Q: Can I view or repeat previous faults?
A: Not at this stage. Handled faults cannot be revisited.

Q: Are the sensors live or simulated?
A: Both modes are supported. Simulation streams realistic data; live mode uses real sensors.

Q: How often is the leaderboard updated?
A: The leaderboard updates after each completed challenge.

About the CIM & Robotics Laboratory:
The CIM & Robotics Laboratory at Braude College of Engineering, established in 1997, offers hands-on experience in industrial robotics, automation, CNC machining, CAD/CAM tools, vision-based quality control, and PLCs. It bridges theory with real-world manufacturing skills.

About the Application:
OptiLine Dashboard is accessible from any device with secure login, supporting both live and simulated data. It provides engineers with real-time monitoring, analysis, troubleshooting, and a gamified performance experience.
"""


In [ ]:
"""
Live Data Helper Functions
--------------------------
Provides access to real-time sensor readings and leaderboard data.
All helper functions are documented and return data in a format suitable for inclusion in a Gemini prompt.
"""

def get_sensor_value(sensor_type: str, location: str, mode: str = "simulation"):
    """
    Retrieves the latest value for a specified sensor type and location.

    Args:
        sensor_type (str): Type of sensor ('temperature', 'humidity', 'pressure', 'dlight', 'distance')
        location (str): 'indoor' or 'outdoor'
        mode (str): 'simulation' or 'mqtt' (default is 'simulation')

    Returns:
        dict: {
            'value': float or None,
            'units': str,
            'timestamp': str (current time, ISO format),
            'error': str (if sensor or location not found)
        }
    """
    # Sensor availability check
    if sensor_type not in SENSORS_META:
        return {'value': None, 'units': '', 'timestamp': '', 'error': f"Sensor '{sensor_type}' not found."}
    if location not in SENSORS_META[sensor_type]['locations']:
        return {'value': None, 'units': '', 'timestamp': '', 'error': f"Sensor '{sensor_type}' is not available in {location}."}

    # Data source selection
    sensor_data = None
    try:
        if location == "indoor":
            data = next(indoor.get_live_data_stream(mode=mode))
        elif location == "outdoor":
            data = next(outdoor.get_live_data_stream(mode=mode))
        else:
            return {'value': None, 'units': '', 'timestamp': '', 'error': "Invalid location."}
        # Try matching by label/case-insensitive
        for key in data.keys():
            if key.lower() == sensor_type.lower():
                sensor_data = data[key]
                break
    except Exception as e:
        return {'value': None, 'units': '', 'timestamp': '', 'error': f"Error retrieving sensor data: {e}"}

    # Return structure
    if sensor_data is not None:
        return {
            'value': sensor_data,
            'units': SENSORS_META[sensor_type]['units'],
            'timestamp': datetime.now().isoformat(timespec="seconds"),
            'error': ""
        }
    else:
        return {
            'value': None,
            'units': SENSORS_META[sensor_type]['units'],
            'timestamp': '',
            'error': f"No data found for sensor '{sensor_type}' at '{location}'."
        }

def get_leaderboard_snapshot(username: str = ""):
    """
    Retrieves the leaderboard: top 5 users and the rank/score of a given user (if specified).

    Args:
        username (str): Username to highlight in the leaderboard (optional).

    Returns:
        dict: {
            'top5': list of (rank, username, score),
            'user_rank': (rank, score) or None
        }
    """
    try:
        top5, user_info = uc.get_leaderboard(username)
        return {
            "top5": top5,
            "user_rank": user_info  # (rank, score) tuple or None
        }
    except Exception as e:
        return {
            "top5": [],
            "user_rank": None,
            "error": f"Leaderboard error: {e}"
        }


In [ ]:
"""
Main Entry Point: ask_optibot
-----------------------------
Handles incoming user questions, enriches them with live data when relevant, builds a context-aware prompt for Gemini,
and returns a natural English answer.

All responses are in English only.
"""

def ask_optibot(message: str, chat_history: list, sensor_mode: str = "simulation") -> tuple:
    """
    Main entry point for OptiBot Q&A.

    Args:
        message (str): User's latest message (question), in English.
        chat_history (list): List of previous messages: {"role": "user"/"assistant", "content": str}
        sensor_mode (str): 'simulation' or 'mqtt' (default: simulation)

    Returns:
        tuple:
            - Empty string (to clear Gradio input)
            - Updated chat_history (with the new user message and the bot response)
    """
    if chat_history is None:
        chat_history = []

    chat_history.append({"role": "user", "content": message})
    message_lower = message.lower()
    context_lines = []

    # --- Intent: Live sensor value ---
    # Tries to extract sensor & location (very basic keyword check, relying on Gemini for ambiguity)
    sensor_types = list(SENSORS_META.keys())
    locations = ["indoor", "outdoor"]
    found_sensor = None
    found_location = None
    for sensor in sensor_types:
        if sensor in message_lower:
            found_sensor = sensor
            break
    for loc in locations:
        if loc in message_lower:
            found_location = loc
            break
    # Default if only sensor found: outdoor if available, else indoor
    if found_sensor and not found_location:
        if "outdoor" in SENSORS_META[found_sensor]["locations"]:
            found_location = "outdoor"
        else:
            found_location = "indoor"

    live_data_note = ""
    if found_sensor and found_location:
        data = get_sensor_value(found_sensor, found_location, mode=sensor_mode)
        if data["error"]:
            live_data_note = data["error"]
        else:
            live_data_note = (
                f"Current value of {found_location} {found_sensor}: "
                f"{data['value']} {data['units']} (as of {data['timestamp']})."
            )

    # --- Intent: Leaderboard ---
    elif "leaderboard" in message_lower or "score" in message_lower or "rank" in message_lower:
        # Try to extract username if mentioned (using @ or 'user ...')
        import re
        user_search = re.search(r"(user ?name|@|user)\s*(\w+)", message_lower)
        username = ""
        if user_search:
            username = user_search.group(2)
        # Else, just pass empty username to get_leaderboard_snapshot
        board = get_leaderboard_snapshot(username)
        if "error" in board:
            live_data_note = board["error"]
        else:
            table = ", ".join([f"{rank}. {name} ({score})" for rank, name, score in board["top5"]])
            user_rank = ""
            if board["user_rank"]:
                rank, score = board["user_rank"]
                user_rank = f"Your current rank is {rank} with {score} points."
            live_data_note = f"Leaderboard (top 5): {table}. {user_rank}"

    # --- Intent: Faults ---
    elif "fault" in message_lower or "error" in message_lower:
        try:
            faults = fb.get_active_faults()
            num_faults = len(faults) if faults else 0
            live_data_note = (
                f"There are currently {num_faults} active faults in the system."
                if num_faults else "No active faults found at the moment."
            )
        except Exception as e:
            live_data_note = f"Could not retrieve fault data: {e}"

    # --- Fallback: nothing to add, but always include User Guide, FAQ, Sensors Meta ---
    # The prompt to Gemini includes live_data_note (if available), then OPTI_BOT_CONTEXT

    # Build prompt for Gemini
    prompt = OPTI_BOT_CONTEXT.strip() + "\n"
    if live_data_note:
        prompt += f"\nRelevant live system data:\n{live_data_note}\n"
    prompt += f"\nUser question: {message}\n"

    # Call Gemini API
    try:
        if not GEMINI_API_KEY:
            bot_response = "Error: Gemini API key is missing. Please contact your administrator."
        else:
            genai.configure(api_key=GEMINI_API_KEY)
            model = genai.GenerativeModel("gemini-1.5-flash")
            response = model.generate_content(prompt)
            bot_response = response.text.strip() if hasattr(response, "text") else str(response)
    except Exception as ex:
        bot_response = (
            "Sorry, I could not process your question due to a system error: "
            f"{ex}"
        )

    chat_history.append({"role": "assistant", "content": bot_response})

    return "", chat_history
